<h1>DSCI-552 Homework 7</h1>
<br>
<font size="4">Morgan Gautho | GitHub: morgangauth | 3944-8632-72</font>
    

In [53]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from scipy.spatial import distance


import warnings
warnings.filterwarnings('ignore')

##  1. Multi-class and Multi-Label Classification Using Support Vector Machines

### (a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.

In [13]:
allData = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")

train, test = train_test_split(allData, test_size=0.3, train_size = 0.7)
x_train = train.iloc[:,:-4]
x_test = test.iloc[:,:-4]
y_train = train.iloc[:,-4:-1]
y_test = test.iloc[:,-4:-1]
y_train.head()


,Family,Genus,Species
1228,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1299,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
3793,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5929,Hylidae,Hypsiboas,HypsiboasCordobae
4331,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


### (b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance). We first try this approach:

#### i. . Research exact match and hamming score/ loss methods for evaluating multilabel classification and use them in evaluating the classifiers in this problem.

- Exact Match is a classification metric in which only samples which have ALL labels clasified correctly are considered "correct". It does not distinguish between partially correct or fully incorrect samples, therefore it is very strict and can be overly harsh.

- Hamming loss reports how many times, on average, the relevance of an example to a class label is incorrrectly predicted. Meaning, it takes into account prediction error and missing error normalized over total classes and number of examples

#### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation.1 You are welcome to try to solve the problem with both standardized 2 and raw attributes and report the results.


In [22]:
c_vals = np.linspace(.001,10000,10)
gamma_vals = np.linspace(.001,3,10)
params = {'C': c_vals, 'gamma' : gamma_vals}

# Family label
y_train_f = y_train["Family"]
svclassifier_f = SVC(kernel='rbf', decision_function_shape = 'ovr')
gridSearchCv_f = GridSearchCV(svclassifier_f, params , cv=10)
gridSearchCv_f.fit(x_train, y_train_f)
best_f = gridSearchCv_f.best_params_
print('Best Parameters for Family: C = %f, Gamma = %f'%(best_f['C'],best_f['gamma']))

# Genus label
y_train_g = y_train["Genus"]
svclassifier_g = SVC(kernel='rbf', decision_function_shape = 'ovr')
gridSearchCv_g = GridSearchCV(svclassifier_g, params , cv=10)
gridSearchCv_g.fit(x_train, y_train_g)
best_g = gridSearchCv_g.best_params_
print('Best Parameters for Genus: C = %f, Gamma = %f'%(best_g['C'],best_g['gamma']))

# Species label
y_train_s = y_train["Species"]
svclassifier_s = SVC(kernel='rbf', decision_function_shape = 'ovr')
gridSearchCv_s = GridSearchCV(svclassifier_s, params , cv=10)
gridSearchCv_s.fit(x_train, y_train_s)
best_s = gridSearchCv_s.best_params_
print('Best Parameters for Species: C = %f, Gamma = %f'%(best_s['C'],best_s['gamma']))

Best Parameters for Family: C = 1111.112000, Gamma = 2.000333
Best Parameters for Family: C = 1111.112000, Gamma = 2.000333
Best Parameters for Family: C = 1111.112000, Gamma = 1.667111


In [47]:
# Predictions
predictions = pd.DataFrame()
predictions['Family'] = gridSearchCv_f.predict(x_test)
predictions['Genus'] = gridSearchCv_g.predict(x_test)
predictions['Species'] = gridSearchCv_s.predict(x_test)

 
accuracy_f = accuracy_score(y_test['Family'], predictions['Family'])
accuracy_g = accuracy_score(y_test['Genus'], predictions['Genus'])
accuracy_s = accuracy_score(y_test['Species'], predictions['Species'])
hamming_f = hamming_loss(y_test['Family'], predictions['Family'])
hamming_g = hamming_loss(y_test['Genus'], predictions['Genus'])
hamming_s = hamming_loss(y_test['Species'], predictions['Species'])
dict = [{"class":"Family", "accuracyScore":accuracy_f,"hammingLoss":hamming_f},
        {"class":"Genus", "accuracyScore":accuracy_g,"hammingLoss":hamming_g},
        {"class":"Species", "accuracyScore":accuracy_s,"hammingLoss":hamming_s}]
dict = pd.DataFrame.from_dict(dict)

display(dict)

,class,accuracyScore,hammingLoss
0,Family,0.995368,0.004632
1,Genus,0.993979,0.006021
2,Species,0.993979,0.006021


#### iii. Repeat 1(b)ii with L1-penalized SVMs.3 Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.


In [38]:
# standardized data
scaler = preprocessing.StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [40]:
c_vals = np.logspace(-5,8,10)
gamma_vals = np.linspace(.001,3,10)
params = {'C': c_vals}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)

# Family label
gridSearchCv_f2 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_f2.fit(x_train_scaled, y_train_f)
best_f2 = gridSearchCv_f2.best_params_
best_fscore = gridSearchCv_f2.best_score_
print('Best Parameters for L1-penalized SVMs for Family: C = %f, Gamma = %f'%(best_f2['C'],best_fscore))

# Genus label
gridSearchCv_g2 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_g2.fit(x_train_scaled, y_train_g)
best_g2 = gridSearchCv_g2.best_params_
best_gscore = gridSearchCv_g2.best_score_
print('Best Parameters for L1-penalized SVMs for Genus: C = %f, Gamma = %f'%(best_g2['C'],best_gscore))

# Species label
gridSearchCv_s2 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_s2.fit(x_train_scaled, y_train_s)
best_s2 = gridSearchCv_s2.best_params_
best_sscore = gridSearchCv_s2.best_score_
print('Best Parameters for L1-penalized SVMs for Species: C = %f, Gamma = %f'%(best_s2['C'],best_sscore))

Best Parameters for L1-penalized SVMs for Family: C = 5.994843, Gamma = 0.936259
Best Parameters for L1-penalized SVMs for Genus: C = 166.810054, Gamma = 0.951550
Best Parameters for L1-penalized SVMs for Species: C = 5.994843, Gamma = 0.958500


In [48]:
# Predictions
predictions2 = pd.DataFrame()
predictions2['Family'] = gridSearchCv_f2.predict(x_test_scaled)
predictions2['Genus'] = gridSearchCv_g2.predict(x_test_scaled)
predictions2['Species'] = gridSearchCv_s2.predict(x_test_scaled)

 
accuracy_f2 = accuracy_score(y_test['Family'], predictions2['Family'])
accuracy_g2 = accuracy_score(y_test['Genus'], predictions2['Genus'])
accuracy_s2 = accuracy_score(y_test['Species'], predictions2['Species'])
hamming_f2 = hamming_loss(y_test['Family'], predictions2['Family'])
hamming_g2 = hamming_loss(y_test['Genus'], predictions2['Genus'])
hamming_s2 = hamming_loss(y_test['Species'], predictions2['Species'])
dict2 = [{"class":"Family L1 SVM", "accuracyScore":accuracy_f2,"hammingLoss":hamming_f2},
        {"class":"Genus L1 SVM", "accuracyScore":accuracy_g2,"hammingLoss":hamming_g2},
        {"class":"Species L1 SVM", "accuracyScore":accuracy_s2,"hammingLoss":hamming_s2}]
dict2 = pd.DataFrame.from_dict(dict2)

display(dict2)

,class,accuracyScore,hammingLoss
0,Family L1 SVM,0.933766,0.066234
1,Genus L1 SVM,0.945345,0.054655
2,Species L1 SVM,0.953682,0.046318


#### iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [43]:
# SMOTE
sm = SMOTE()
x_res_f, y_res_f = sm.fit_resample(x_train_scaled, y_train['Family'])
x_res_g, y_res_g = sm.fit_resample(x_train_scaled, y_train['Genus'])
x_res_s, y_res_s = sm.fit_resample(x_train_scaled, y_train['Species'])


In [45]:
c_vals = np.logspace(-5,8,10)
gamma_vals = np.linspace(.001,3,10)
params = {'C': c_vals}
l1_svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)

# Family label
gridSearchCv_f3 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_f3.fit(x_res_f, y_res_f)
best_f3 = gridSearchCv_f3.best_params_
best_fscore3 = gridSearchCv_f3.best_score_
print('Best Parameters for L1-penalized SVMs with SMOTE for Family: C = %f, Gamma = %f'%(best_f3['C'],best_fscore3))

# Genus label
gridSearchCv_g3 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_g3.fit(x_res_g, y_res_g)
best_g3 = gridSearchCv_g3.best_params_
best_gscore3 = gridSearchCv_g3.best_score_
print('Best Parameters for L1-penalized SVMs with SMOTE for Genus: C = %f, Gamma = %f'%(best_g3['C'],best_gscore3))

# Species label
gridSearchCv_s3 = GridSearchCV(l1_svm, params , cv=10, scoring = "accuracy")
gridSearchCv_s3.fit(x_res_s, y_res_s)
best_s3 = gridSearchCv_s3.best_params_
best_sscore3 = gridSearchCv_s3.best_score_
print('Best Parameters for L1-penalized SVMs with SMOTE for Species: C = %f, Gamma = %f'%(best_s3['C'],best_sscore3))

Best Parameters for L1-penalized SVMs with SMOTE for Family: C = 5.994843, Gamma = 0.948252
Best Parameters for L1-penalized SVMs with SMOTE for Genus: C = 166.810054, Gamma = 0.954150
Best Parameters for L1-penalized SVMs with SMOTE for Species: C = 166.810054, Gamma = 0.960741


In [49]:
# Predictions
predictions3 = pd.DataFrame()
predictions3['Family'] = gridSearchCv_f3.predict(x_test_scaled)
predictions3['Genus'] = gridSearchCv_g3.predict(x_test_scaled)
predictions3['Species'] = gridSearchCv_s3.predict(x_test_scaled)

 
accuracy_f3 = accuracy_score(y_test['Family'], predictions3['Family'])
accuracy_g3 = accuracy_score(y_test['Genus'], predictions3['Genus'])
accuracy_s3 = accuracy_score(y_test['Species'], predictions3['Species'])
hamming_f3 = hamming_loss(y_test['Family'], predictions3['Family'])
hamming_g3 = hamming_loss(y_test['Genus'], predictions3['Genus'])
hamming_s3 = hamming_loss(y_test['Species'], predictions3['Species'])
dict3 = [{"class":"Family L1 SVM w SMOTE", "accuracyScore":accuracy_f3,"hammingLoss":hamming_f3},
        {"class":"Genus L1 SVM w SMOTE", "accuracyScore":accuracy_g3,"hammingLoss":hamming_g3},
        {"class":"Species L1 SVM w SMOTE", "accuracyScore":accuracy_s3,"hammingLoss":hamming_s3}]
dict3 = pd.DataFrame.from_dict(dict3)

display(dict3)

,class,accuracyScore,hammingLoss
0,Family L1 SVM w SMOTE,0.908291,0.091709
1,Genus L1 SVM w SMOTE,0.910607,0.089393
2,Species L1 SVM w SMOTE,0.956461,0.043539


In [50]:
print("Gaussian Kernel SVM:")
display(dict)
print("L1-penalized SVM:")
display(dict2)
print("L1-penalized SVM with SMOTE:")
display(dict3)
print("Part bii (the Gaussian kernel SVM) had the best accuracy score and hamming loss. Since this is the only non-linear model, it can be assumed that the data set is not linearly separable, which is why the first SVM performed the best.")

Gaussian Kernel SVM:


,class,accuracyScore,hammingLoss
0,Family,0.995368,0.004632
1,Genus,0.993979,0.006021
2,Species,0.993979,0.006021


L1-penalized SVM:


,class,accuracyScore,hammingLoss
0,Family L1 SVM,0.933766,0.066234
1,Genus L1 SVM,0.945345,0.054655
2,Species L1 SVM,0.953682,0.046318


L1-penalized SVM with SMOTE:


,class,accuracyScore,hammingLoss
0,Family L1 SVM w SMOTE,0.908291,0.091709
1,Genus L1 SVM w SMOTE,0.910607,0.089393
2,Species L1 SVM w SMOTE,0.956461,0.043539


Part bii (the Gaussian kernel SVM) had the best accuracy score and hamming loss. Since this is the only non-linear model, it can be assumed that the data set is not linearly separable, which is why the first SVM performed the best.


#### v. Extra Practice: Study the Classifier Chain method and apply it to the above problem.
#### vi. Extra Practice: Research how confusion matrices, precision, recall, ROC, and AUC are defined for multi-label classification and compute them for the classifiers you trained in above.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
#### Monte-Carlo Simulation: Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate.

### (a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.

### (b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.

### (c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss between the true labels and the labels assigned by clusters.


In [75]:
allData = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
allX = allData.iloc[:,:-4]
k_vals = range(2,51)
hamming_losses = []

for i in range(1,51):
    print("iteration -> %d : " %(i))
    ch_scores = []
    for k in k_vals:
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(allX)
        labels = kmeans.labels_
        ch = calinski_harabasz_score(allX, labels)
        ch_scores.append(ch)
    best_k = k_vals[ch_scores.index(max(ch_scores))]
    # repeat k means with best k
    print("Best k value for clusters is : ", best_k)
    kmeans = KMeans(n_clusters = best_k).fit(allX)
    allData["cluster"] = kmeans.labels_
    
    for y in range(0,best_k):
        family = allData[allData.cluster == y]['Family'].value_counts().index[0]
        allData.loc[allData.cluster == y, 'Family_Majority'] = family
        genus = allData[allData.cluster == c]['Genus'].value_counts().index[0]
        allData.loc[allData.cluster == y, 'Genus_Majority'] = genus
        species = allData[allData.cluster == c]['Species'].value_counts().index[0]
        allData.loc[allData.cluster == y, 'Species_Majority'] = species
    print("The majority classes are as follows: family: ", family,". genus: ", genus,". species: ", species)
    
    scores = []
    labels = ["Family","Genus","Species"]
    for c in range(0, best_k):
        cluster_scores = []
        cluster = allData[allData.cluster == c]
        for l in labels:
            cluster_label = l+"_Majority"
            score = hamming_loss(cluster[l], cluster[cluster_label])
            cluster_scores.append(score)
            print('Cluster %d >> %s: %f'%(c,l,score))
        avg_hamming_loss = np.mean(cluster_scores)
        scores.append(avg_hamming_loss)
        print('Average for Cluster %d: %f'%(c,avg_hamming_loss))
    mean_loss = np.mean(scores)
    print('Mean hamming loss:',mean_loss)
    hamming_losses.append(mean_loss)
    
    
    
    

iteration -> 1 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas . species:  HypsiboasCordobae
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.996109
Cluster 0 >> Species: 0.996109
Average for Cluster 0: 0.675560
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Mean hamming loss: 0.6187551347655584
iteration -> 2 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas . species:  HypsiboasCordobae
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.996109
Cluster 0 >> Species: 0.996109
Average for Cluster 0: 0.675560
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Mean hamming loss: 0.6187551347655584
iteration -> 3 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboa

Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas . species:  HypsiboasCordobae
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.996109
Cluster 0 >> Species: 0.996109
Average for Cluster 0: 0.675560
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Mean hamming loss: 0.6187551347655584
iteration -> 21 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas . species:  HypsiboasCordobae
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.996109
Cluster 0 >> Species: 0.996109
Average for Cluster 0: 0.675560
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Mean hamming loss: 0.6187551347655584
iteration -> 22 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Leptodactylidae . genus:  Adenomera . spec

Best k value for clusters is :  2
The majority classes are as follows: family:  Leptodactylidae . genus:  Adenomera . species:  AdenomeraHylaedactylus
Cluster 0 >> Family: 0.432583
Cluster 0 >> Genus: 0.810954
Cluster 0 >> Species: 0.997776
Average for Cluster 0: 0.747104
Cluster 1 >> Family: 0.034464
Cluster 1 >> Genus: 0.035575
Cluster 1 >> Species: 0.035575
Average for Cluster 1: 0.035205
Mean hamming loss: 0.39115440579046756
iteration -> 40 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas . species:  HypsiboasCordobae
Cluster 0 >> Family: 0.034464
Cluster 0 >> Genus: 0.996109
Cluster 0 >> Species: 0.996109
Average for Cluster 0: 0.675560
Cluster 1 >> Family: 0.432583
Cluster 1 >> Genus: 0.561023
Cluster 1 >> Species: 0.692244
Average for Cluster 1: 0.561950
Mean hamming loss: 0.6187551347655584
iteration -> 41 : 
Best k value for clusters is :  2
The majority classes are as follows: family:  Hylidae . genus:  Hypsiboas 

In [74]:
print("The mean of the hamming losses is ", np.mean(hamming_losses), " and the standard deviation is ", np.std(hamming_losses))




The mean of the hamming losses is  0.5277148431755222  and the standard deviation is  0.11150113021489183


## 3. ISLR 12.6.2

. Suppose that we have four observations, for which we compute a
dissimilarity matrix, given by <br>
[] 0.3 0.4 0.7 <br>
0.3 [] 0.5 0.8<br>
0.4 0.5 [] 0.45<br>
0.7 0.8 0.45 [] <br>

For instance, the dissimilarity between the first and second observations is 0.3, and the dissimilarity between the second and fourth observations is 0.8.

(a) On the basis of this dissimilarity matrix, sketch the dendrogram
that results from hierarchically clustering these four observations using complete linkage. Be sure to indicate on the plot the
height at which each fusion occurs, as well as the observations
corresponding to each leaf in the dendrogram.

(b) Repeat (a), this time using single linkage clustering.

See image below

In [70]:
from IPython.display import IFrame
IFrame("../IMG_9241.jpg", width=1000, height=1000)

#### (c) Suppose that we cut the dendrogram obtained in (a) such that two clusters result. Which observations are in each cluster?

If the dendogram were cut in half, one cluster would contain observations 1,2 and the other cluster would contain observations 3,4.

#### (d) Suppose that we cut the dendrogram obtained in (b) such that two clusters result. Which observations are in each cluster?

If the dendogram were cut in half, one cluster would contain observations 1,2 in a cluster with 3,  and the other cluster would contain observation 4.

#### (e) It is mentioned in the chapter that at each fusion in the dendrogram, the position of the two clusters being fused can be swapped without changing the meaning of the dendrogram. Draw a dendrogram that is equivalent to the dendrogram in (a), for which two or more of the leaves are repositioned, but for which the meaning of the dendrogram is the same.

In [76]:
from IPython.display import IFrame
IFrame("../IMG_9242.jpg", width=1000, height=1000)

# References

https://mmuratarat.github.io/2020-01-25/multilabel_classification_metrics <br>
https://realpython.com/k-means-clustering-python/ <br>
